<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/12_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline



*   NN.TRANSFORMER 와 TORCHTEXT 로 seq2seq 모델링
===============================================================
출처) https://tutorials.pytorch.kr/


![](https://github.com/pytorch/tutorials/blob/gh-pages/_static/img/transformer_architecture.jpg?raw=1)


nn.TransformerEncoder 모델을 언어 모델링(language modeling) 과제에 적용한 예제임
- 과제는 주어진 단어 (또는 단어의 시퀀스)가 다음에 이어지는 단어 시퀀스를 따를 가능성(likelihood)에 대한 확률을 계산하는 것임
- 먼저, 토큰(token) 들의 시퀀스가 임베딩(embedding) 레이어로 전달되며, 이어서 포지셔널 인코딩(positional encoding) 레이어가 각 단어의 순서를 설명
- nn.TransformerEncoder 는 여러 개의 nn.TransformerEncoderLayer 레이어로 구성되어 있음
- nn.TransformerEncoder 내부의 셀프-어텐션(self-attention) 레이어들은 시퀀스 안에서의 이전 포지션에만 집중하도록 허용되기 때문에, 입력(input) 순서와 함께, 어텐션 마스크(attention mask) 가 필요함
- 언어 모델링 과제를 위해서, 다음의 포지션에 있는 모든 토큰들은 마스킹 되어야(가려져야) 함. 실제 단어를 얻기 위해서, nn.TransformerEncoder 의 출력은 로그-소프트맥스(log-Softmax) 로 이어지는 최종 선형(Linear) 레이어로 전달됨.





모델정의
----------------




In [2]:
import math
import torch
import torch.nn as nn  # 파이토치의 신경망 패키지
import torch.nn.functional as F # 활성화 함수, 손실 함수 등을 포함하는 모듈 (관례에 따라 일반적으로 F 네임스페이스로 임포트 됩니다) 

# nn.Module(자체가 클래스이고 상태를 추척할 수 있는) 파라미터로 사용할 수 있는 TransformerModel 클래스 구성

# ntokens = len(TEXT.vocab.stoi) # 단어 사전의 크기
# emsize(ninp) = 200 # 임베딩 차원
# nhead = 2 # 멀티헤드 어텐션(multi-head attention) 모델의 헤드 개수
# nhid = 200 # nn.TransformerEncoder 에서 피드포워드 네트워크(feedforward network) 모델의 차원
# nlayers = 2 # nn.TransformerEncoder 내부의 nn.TransformerEncoderLayer 개수
# dropout # 드랍아웃(dropout) 값

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

# PositionalEncoding 모듈
- 시퀀스 안에서 토큰의 상대적인 또는 절대적인 포지션에 대한 정보를 생성 
- 포지셔널 인코딩은 임베딩과 합칠 수 있도록 똑같은 차원으로 구성
- 다른 주파수(frequency) 의 sine 과 cosine 함수를 사용함




In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model) # 스칼라 값으로 채워진 텐서를 반환
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

데이터 로드하고 배치 만들기
-------------------




학습 과정에서는 torchtext 의 Wikitext-2 데이터셋을 이용

단어 오브젝트는 훈련 데이터셋(train dataset) 에 의하여 만들어지고, 토큰을 텐서(tensor)로 수치화하는데 사용

시퀀스 데이터로부터 시작하여, batchify() 함수는 데이터셋을 컬럼들로 배열하고, batch_size 사이즈의 배치들로 나눈 후에 남은 모든 토큰을 버림

예를 들어, 알파벳을 시퀀스(총 길이 26) 로 생각하고 배치 사이즈를 4라고 한다면, 우리는 알파벳을 길이가 6인 4개의 시퀀스로 나눌 수 있음

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}

이 컬럼들은 모델에 의해서 독립적으로 취급되며, 이것은 더 효율적인 배치 프로세싱(batch processing) 이 가능하지만, G 와 F 의 의존성이 학습될 수 없다는 것을 의미함




In [4]:
!pip3 install torchtext==0.4.0

     |████████████████████████████████| 53 kB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [5]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 17.1MB/s]


extracting


# 입력(input) 과 타겟(target) 시퀀스를 생성하기 위한 함수들

get_batch() 함수는 트랜스포머 모델을 위한 입력과 타겟 시퀀스를 생성합

![](https://github.com/pytorch/tutorials/blob/gh-pages/_static/img/transformer_input_target.png?raw=1)


변수는 트랜스포머 모델의 S 차원과 일치하는 0 차원에 해당합니다. 배치 차원 N 은 1 차원에 해당합니다.




In [6]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

인스턴스(instance) 초기화하기
--------------------




모델은 아래와 같은 하이퍼파라미터(hyperparameter) 로 세팅 

In [7]:
ntokens = len(TEXT.vocab.stoi) # 단어 사전의 크기
emsize = 200 # 임베딩 차원
nhid = 200 # nn.TransformerEncoder 에서 피드포워드 네트워크(feedforward network) 모델의 차원
nlayers = 2 # nn.TransformerEncoder 내부의 nn.TransformerEncoderLayer 개수
nhead = 2 # 멀티헤드 어텐션(multi-head attention) 모델의 헤드 개수
dropout = 0.2 # 드랍아웃(dropout) 값
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

모델 실행하기
-------------




손실(loss) 을 추적하는 데에는 CrossEntropyLoss 가 적용되며, 옵티마이저(optimizer) 로서 SGD 는 확률적 경사 하강법(stochastic gradient descent method) 을 구현합니다. 초기 학습률(learning rate) 은 5.0 으로 설정됩니다. StepLR 은 에포크(epoch) 에 따라서 학습률을 조절하는데 사용됩니다. 학습하는 동안에, 우리는 기울기 폭발(gradient exploding) 을 방지하기 위하여 모든 기울기를 함께 스케일(scale) 하는 함수인 nn.utils.clip_grad_norm_ 을 이용합니다.



In [8]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # 학습률
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # 학습 모드를 시작합니다.
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # 평가 모드를 시작합니다.
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

에포크 내에서 반복됩니다. 만약 검증 오차(validation loss) 가 우리가 지금까지 관찰한 것 중 최적이라면 모델을 저장합니다. 매 에포크 이후에 학습률을 조절합니다.



In [ ]:
best_val_loss = float("inf")
epochs = 10 # The number of epochs (테스트를 위해서 최소 수준으로 조정)
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:372: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   200/ 2981 batches | lr 5.00 | ms/batch 38.74 | loss  8.05 | ppl  3137.49
| epoch   1 |   400/ 2981 batches | lr 5.00 | ms/batch 37.02 | loss  6.78 | ppl   876.03
| epoch   1 |   600/ 2981 batches | lr 5.00 | ms/batch 35.86 | loss  6.36 | ppl   576.50
| epoch   1 |   800/ 2981 batches | lr 5.00 | ms/batch 36.06 | loss  6.23 | ppl   506.14
| epoch   1 |  1000/ 2981 batches | lr 5.00 | ms/batch 36.12 | loss  6.11 | ppl   451.33
| epoch   1 |  1200/ 2981 batches | lr 5.00 | ms/batch 39.85 | loss  6.08 | ppl   438.19
| epoch   1 |  1400/ 2981 batches | lr 5.00 | ms/batch 35.98 | loss  6.04 | ppl   420.44
| epoch   1 |  1600/ 2981 batches | lr 5.00 | ms/batch 36.10 | loss  6.05 | ppl   422.99
| epoch   1 |  1800/ 2981 batches | lr 5.00 | ms/batch 36.17 | loss  5.95 | ppl   383.86
| epoch   1 |  2000/ 2981 batches | lr 5.00 | ms/batch 36.76 | loss  5.96 | ppl   385.90
| epoch   1 |  2200/ 2981 batches | lr 5.00 | ms/batch 36.00 | loss  5.85 | ppl   346.27
| epoch   1 |  2400/ 

평가 데이터셋(test dataset) 으로 모델을 평가하기
-------------------------------------

평가 데이터셋에 대한 결과를 확인하기 위해서 최고의 모델을 적용합니다.



In [ ]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)